In [1]:
import sys
sys.path.append('../../..')
import pandas as pd
from torch.utils import data
import torch
import re
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from torch.optim.lr_scheduler import LambdaLR
import matplotlib.pyplot as plt
from tqdm import tqdm,trange
import matplotlib
from datetime import datetime
import copy
import numpy as np
import random
from dataloader.dataloader import MyData
from model.Decoder_Only import Decoder_Only
import os
from torch.cuda.amp import autocast, GradScaler

/data/miniconda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(epoch,batch_size,lr,learning_de,de_step,train_dataset,model,scaler):
    train_dataloader=DataLoader(train_dataset,batch_size=int(batch_size),shuffle=True,drop_last=True)
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    
    with autocast():
        for i in range(epoch):
            print("now_learning_rate:"+str(optimizer.param_groups[0]["lr"]))
            with tqdm(total=len(train_dataloader)) as _tqdm: # 使用需要的参数对tqdm进行初始化   
                for data in train_dataloader:
                    tokenizer_output=model.use_tokenizer(data)
                    prediction=model.get_ans(tokenizer_output['input_ids'].to(os.environ.get('device')))
                
                    ls=model.loss(prediction,tokenizer_output['input_ids'].to(os.environ.get('device')),tokenizer_output['attention_mask'].to(os.environ.get('device')))
                    _tqdm.set_postfix(loss='{:.3f}'.format(ls)) # 设置你想要在本次循环内实时监视的变量  可以作为后缀打印出来
                    _tqdm.update(1)
                    scaler.scale(ls).backward()
                    scaler.step(optimizer)
                    scaler.update()

                    optimizer.zero_grad() 
        torch.save(model.state_dict(), 'model')
        if (i+1)%de_step==0:
            optimizer.param_groups[0]["lr"]=optimizer.param_groups[0]["lr"]*learning_de

In [3]:
vocab_size=73448
epoch=1
batch_size=16
lr=1e-4
de=0.5
de_step=5
os.environ['device']='cuda'


num_layer=6
num_header=12
input_length=512
embeding_dim=512
use_group_attention=True
group_num=3

model=Decoder_Only(
    num_header=num_header,
    num_layer=num_layer,
    embeding_dim=embeding_dim,
    input_length=input_length,
    vocab_size=vocab_size,
    use_group_attention=use_group_attention,
    group_num=group_num,
    use_check_point=True
    ).to(os.environ.get('device'))

train_dataset=MyData()
scaler = GradScaler()

2025-02-27 23:28:38,585 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-02-27 23:28:38,966 - modelscope - INFO - Target directory already exists, skipping creation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_42178/1219914656.py:29: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
train(epoch,batch_size,lr,de,de_step,train_dataset,model,scaler)

/tmp/ipykernel_42178/78680193.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


now_learning_rate:0.0001


  0%|          | 1/62888 [00:00<15:26:01,  1.13it/s, loss=5.331]/data/miniconda/envs/torch/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  0%|          | 237/62888 [05:43<25:14:07,  1.45s/it, loss=3.064]